1. Read the rows in the json file

In [ ]:
import json
def run(file_name:str, print_lines:bool=False) -> None:
    """
    From JSON row file read each row

    Args:
        file_name (str): file path to read
        print_lines (bool): print lines from file to console.
    """
    # track the row count
    line_num = 1        # Current line number
    ok_count = 0        # Rows that didn't return an error
    reject_count = 0    # Rows that did return an error
    with open(file_name, "r") as json_file:
        for line in json_file:
            try:
                # read lines into a dict
                row = json.loads(line.strip())
                # Print lines
                if print_lines:
                    print(f"[{line_num:04d} [OK]: {row}")
                # line count update
                ok_count += 1
            except Exception as err:
                # print row with error
                print(f"[{line_num:04d}][ERROR:] {str(err)}, [DATA]: {row}")
                reject_count += 1
            finally:
                # Update line count regardless
                line_num += 1
        # Print line count summary
    print(f"Read {line_num - 1} rows")
    print(f"OK rows: {ok_count:04d}, Rejected rows: {reject_count:04d}") 

# Call the function for a test
file_path = "./data/vehicles_complex.json"
run(file_path, print_lines=True)

2. Reject any rows that does not meet any of the requirements below:
   1. missing any of the following fields: _license plate, make and model, year, registered name, date or address_
   2. Make sure any of the above fields don't include Nulls. 
   3. **BUT** if you're missing the registered name and you do have sales records, you can use the name with the most recent sale date and use that as the registered name. You don't reject the row in this case.
   4. Make sure all the addresses are valid. Reject them if they are empty or invalid.

In [ ]:
# later make it date OR address
VALID_FIELDS = ["license_plate", "make_model", "year", "registered_name", "registered_address", "registered_date"]